In [1]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize
%matplotlib inline

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm import tqdm
from robust_optimization import RobustOptimizer
import copy
import timm
import torch.utils.data as data

print(f"Torch: {torch.__version__}")

Torch: 1.10.2


C:\Users\Darkl\anaconda3\envs\mtcnn\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Darkl\anaconda3\envs\mtcnn\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Training settings
batch_size = 32
epochs = 40
lr = 3e-5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

training_size = 28317 + 3541
IMG_SIZE=260
image_path = '../dataset/images.npy'
training_emotion_path = "../dataset/emotions_1.npy"
test_emotion_path = '../dataset/emotions_multi.npy'

True


In [3]:
'''
def load_data(image_path, training_emotion_path, subset):
    images = np.load(image_path)        # shape = (35393, 48, 48, 1)
    # images = images/255.0
    images = np.float32(images)
    
    if subset == 'train':
        training_emotions = np.load(training_emotion_path)    # shape = (35393, 8)
        training_emotions = np.float32(training_emotions)
        return images[:training_size], training_emotions[:training_size]
    if subset == 'test':
        test_emotions = np.load(test_emotion_path)
        test_emotions = np.float32(test_emotions)
        return images[training_size:], test_emotions[training_size:]
'''
sample_size = 35393
training_size = 28317 + 3541
emotions = np.load(test_emotion_path)
test_subset_indices = []
for i in range(sample_size):
    if np.count_nonzero(emotions[i] > 0) >= 4:
        test_subset_indices.append(i)
test_subset_indices = random.sample(test_subset_indices, k=sample_size-training_size)
train_subset_indices = [item for item in range(sample_size) if not item in test_subset_indices]

def load_data(image_path, training_emotion_path, subset):
    images = np.load(image_path)   
    images = np.float32(images)
    if subset == 'train':
        training_emotions = np.load(training_emotion_path) 
        training_emotions = np.float32(training_emotions)
        return images[train_subset_indices], training_emotions[train_subset_indices]
    if subset == 'test':
        test_emotions = np.load(test_emotion_path)
        test_emotions = np.float32(test_emotions)
        return images[test_subset_indices], test_emotions[test_subset_indices]
    
def img_proc(img):
    img = torch.tensor(img)                             # (48, 48, 1)
    img = torch.reshape(img, (1, 48, 48))               # (1, 48, 48)
    img = transforms.Resize([IMG_SIZE, IMG_SIZE])(img)  # (1, 260, 260)
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img = img.repeat(3, 1, 1)                           # (3, 260, 260)
    return img

class FERPlusDataset(data.Dataset):
    def __init__(self, image_path, training_emotion_path, subset):
        assert(subset=='train' or subset=='test')
        self.images, self.emotions = load_data(image_path, training_emotion_path, subset)

    def __getitem__(self, index):
        image = self.images[index]
        emotion = self.emotions[index]

        return img_proc(image), emotion

    def __len__(self):
        return len(self.images)

train_loader = torch.utils.data.DataLoader(
    FERPlusDataset(
        image_path,
        training_emotion_path,
        'train'
    ),
    batch_size=batch_size,
    shuffle=True,
)

val_loader = torch.utils.data.DataLoader(
    FERPlusDataset(
        image_path,
        training_emotion_path,
        'test'
    ),
    batch_size=batch_size,
    shuffle=False,
)

def accuracy(output, target):
    batch_size = target.size(0)
    acc = 0
    for i in range(batch_size):
        true = target[i]
        pred = output[i]
        index_max = torch.argmax(pred)
        if true[index_max] == torch.max(true):
            acc += 1
    acc = float(acc)/batch_size
    return acc

In [4]:
def train(model,n_epochs=epochs, learningrate=lr, robust=False):
    # optimizer
    if robust:
        optimizer = RobustOptimizer(filter(lambda p: p.requires_grad, model.parameters()), optim.Adam, lr=learningrate)
    else:
        optimizer=optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learningrate)

    best_acc=0
    best_model=None
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        model.train()
        for img, label in tqdm(train_loader):
            img = img.to(device)
            label = label.to(device)

            output = model(img)
            loss = criterion(output, label)

            if robust:
                #optimizer.zero_grad()
                loss.backward()
                optimizer.first_step(zero_grad=True)
  
                # second forward-backward pass
                output = model(img)
                loss = criterion(output, label)
                loss.backward()
                optimizer.second_step(zero_grad=True)
            else:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            acc = (output.argmax(dim=1) == label.argmax(dim=1)).float().mean()
            epoch_accuracy += acc / len(train_loader)
            epoch_loss += loss / len(train_loader)

        model.eval()
        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for img, label in val_loader:
                img = img.to(device)
                label = label.to(device)

                val_output = model(img)
                val_loss = criterion(val_output, label)

                acc = accuracy(val_output, label)
                epoch_val_accuracy += acc / len(val_loader)
                epoch_val_loss += val_loss / len(val_loader)

        print(
            f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
        if best_acc<epoch_val_accuracy:
            best_acc=epoch_val_accuracy
            best_model=copy.deepcopy(model.state_dict())
        #scheduler.step()
    
    if best_model is not None:
        model.load_state_dict(best_model)
        print(f"Best acc:{best_acc}")
        model.eval()
        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for img, label in val_loader:
                img = img.to(device)
                label = label.to(device)

                val_output = model(img)
                val_loss = criterion(val_output, label)

                acc = accuracy(val_output, label)
                epoch_val_accuracy += acc / len(val_loader)
                epoch_val_loss += val_loss / len(val_loader)

        print(
            f"val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
    else:
        print(f"No best model Best acc:{best_acc}")

In [5]:
def set_parameter_requires_grad(model, requires_grad):
    for param in model.parameters():
        param.requires_grad = requires_grad

In [6]:
criterion = nn.MSELoss().to(device) 
model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))

model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

In [7]:
set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:47<00:00, 20.83it/s]


Epoch : 1 - loss : 0.0585 - acc: 0.7440 - val_loss : 0.0343 - val_acc: 0.5503



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:45<00:00, 21.83it/s]


Epoch : 2 - loss : 0.0523 - acc: 0.7785 - val_loss : 0.0331 - val_acc: 0.5501



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:44<00:00, 22.27it/s]


...

Epoch : 8 - loss : 0.0499 - acc: 0.7938 - val_loss : 0.0350 - val_acc: 0.5687



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:44<00:00, 22.18it/s]


Epoch : 9 - loss : 0.0502 - acc: 0.7944 - val_loss : 0.0337 - val_acc: 0.5661



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:44<00:00, 22.15it/s]


Epoch : 10 - loss : 0.0501 - acc: 0.7970 - val_loss : 0.0392 - val_acc: 0.5527

Best acc:0.5689564564564563
val_loss : 0.0375 - val_acc: 0.5690



In [8]:
set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

torch.save(model.state_dict(), './models/mtcnn-1label-cplx.pth')

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:35<00:00,  6.40it/s]


Epoch : 1 - loss : 0.0385 - acc: 0.8374 - val_loss : 0.0254 - val_acc: 0.6409



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:33<00:00,  6.48it/s]


Epoch : 2 - loss : 0.0295 - acc: 0.8769 - val_loss : 0.0265 - val_acc: 0.6570



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:33<00:00,  6.48it/s]


...

Epoch : 28 - loss : 0.0011 - acc: 0.9965 - val_loss : 0.0498 - val_acc: 0.6547



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:34<00:00,  6.46it/s]


Epoch : 29 - loss : 0.0012 - acc: 0.9953 - val_loss : 0.0497 - val_acc: 0.6520



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:36<00:00,  6.35it/s]


Epoch : 30 - loss : 0.0012 - acc: 0.9957 - val_loss : 0.0484 - val_acc: 0.6462

Best acc:0.6772522522522525
val_loss : 0.0394 - val_acc: 0.6773



In [9]:
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_2.npy"

model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))
model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

torch.save(model.state_dict(), './models/mtcnn-2label-cplx.pth')

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:49<00:00, 20.27it/s]


Epoch : 1 - loss : 0.0587 - acc: 0.7417 - val_loss : 0.0362 - val_acc: 0.5408



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:46<00:00, 21.50it/s]


Epoch : 2 - loss : 0.0520 - acc: 0.7816 - val_loss : 0.0354 - val_acc: 0.5447



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:47<00:00, 21.09it/s]


...

Epoch : 28 - loss : 0.0013 - acc: 0.9953 - val_loss : 0.0480 - val_acc: 0.6559



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:53<00:00,  5.73it/s]


Epoch : 29 - loss : 0.0012 - acc: 0.9961 - val_loss : 0.0475 - val_acc: 0.6590



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:44<00:00,  6.05it/s]


Epoch : 30 - loss : 0.0010 - acc: 0.9966 - val_loss : 0.0474 - val_acc: 0.6668

Best acc:0.6780593093093098
val_loss : 0.0370 - val_acc: 0.6781



In [10]:
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_3.npy"

model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))
model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

torch.save(model.state_dict(), './models/mtcnn-3label-cplx.pth')

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:48<00:00, 20.70it/s]


Epoch : 1 - loss : 0.0585 - acc: 0.7443 - val_loss : 0.0349 - val_acc: 0.4898



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:48<00:00, 20.62it/s]


Epoch : 2 - loss : 0.0519 - acc: 0.7842 - val_loss : 0.0328 - val_acc: 0.5385



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:48<00:00, 20.65it/s]


Epoch : 3 - loss : 0.0508 - acc: 0.7882 - val_loss : 0.0330 - val_acc: 0.5357



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:48<00:00, 20.43it/s]


KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_4.npy"

model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))
model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

torch.save(model.state_dict(), './models/mtcnn-4label-cplx.pth')

In [ ]:
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_5.npy"

model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))
model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

torch.save(model.state_dict(), './models/mtcnn-5label-cplx.pth')

In [ ]:
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_6.npy"

model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))
model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

torch.save(model.state_dict(), './models/mtcnn-6label-cplx.pth')

In [ ]:
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_7.npy"

model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))
model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

torch.save(model.state_dict(), './models/mtcnn-7label-cplx.pth')

In [ ]:
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_multi.npy"

model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))
model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

torch.save(model.state_dict(), './models/mtcnn-8label-cplx.pth')